<a href="https://colab.research.google.com/github/MelaOchoa/LosCapis/blob/Taller2_ModelosSupervisado/RegresionPolinomica_LosCapis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from numpy import hstack
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
datos = pd.read_csv("APY.csv")

In [ ]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345336 entries, 0 to 345335
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   State       345336 non-null  object 
 1   District    345336 non-null  object 
 2   Crop        345327 non-null  object 
 3   Crop_Year   345336 non-null  int64  
 4   Season      345336 non-null  object 
 5   Area        345336 non-null  float64
 6   Production  340388 non-null  float64
 7   Yield       345336 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 21.1+ MB


In [ ]:
datos.head()

,State,District,Crop,Crop_Year,Season,Area,Production,Yield
0,Andaman and Nicobar Island,NICOBARS,Arecanut,2007,Kharif,2439.6,3415.0,1.40
1,Andaman and Nicobar Island,NICOBARS,Arecanut,2007,Rabi,1626.4,2277.0,1.40
2,Andaman and Nicobar Island,NICOBARS,Arecanut,2008,Autumn,4147.0,3060.0,0.74
3,Andaman and Nicobar Island,NICOBARS,Arecanut,2008,Summer,4147.0,2660.0,0.64
4,Andaman and Nicobar Island,NICOBARS,Arecanut,2009,Autumn,4153.0,3120.0,0.75


In [ ]:
datos = datos.drop('Yield', axis=1)

In [ ]:
# Eliminar filas con valores NaN en cualquier columna
datos = datos.dropna().reset_index(drop=True)

In [ ]:
datos.head()

,State,District,Crop,Crop_Year,Season,Area,Production
0,Nagaland,PHEK,Tapioca,2017,Kharif,180.0,3650.0
1,Andhra Pradesh,RANGAREDDI,Banana,2005,Whole Year,40.0,836.0
2,Assam,JORHAT,Sweet potato,2011,Whole Year,74.0,244.0
3,Jammu and Kashmir,UDHAMPUR,Rapeseed &Mustard,2006,Rabi,2833.0,2079.0
4,Nagaland,PHEK,Tapioca,2019,Kharif,200.0,4052.0


In [ ]:
datos = datos.sample(n=1000, random_state=42).reset_index(drop=True)

In [ ]:
# Características (X) y etiqueta (y)
X_categoricas = datos[["State", "District ", "Crop", "Season"]]
X_numericas = datos[["Crop_Year", "Area "]]
y = datos["Production"].values.reshape(-1, 1)

# Codificación OneHot para variables categóricas
encoder = OneHotEncoder(drop='first', sparse_output=False)
X_categoricas_procesada = encoder.fit_transform(X_categoricas)

# Combinar variables categóricas codificadas y numéricas
X_procesada = hstack([X_categoricas_procesada, X_numericas])

# Transformar las características para regresión polinomial (grado 2)
grado = 2  # Puedes ajustar el grado aquí
poly = PolynomialFeatures(degree=grado, include_bias=False)
X_poly = poly.fit_transform(X_procesada)

# Crear el modelo de regresión polinomial con Ridge (para evitar sobreajuste)
modelo = make_pipeline(StandardScaler(), Ridge(alpha=1.0))
modelo.fit(X_poly, y)

Pipeline(steps=[('standardscaler', StandardScaler()), ('ridge', Ridge())])

In [ ]:
state = "Gujarat"
district = "AHMADABAD"
crop = "Maize"
season = "Rabi       "
crop_year = 2013
area = 320
# Crear un DataFrame con los valores fijos
nuevos_datos = pd.DataFrame({
    "State": [state],
    "District ": [district],
    "Crop": [crop],
    "Season": [season],
    "Crop_Year": [crop_year],
    "Area ": [area]
})

# Codificación OneHot para los nuevos datos
X_categoricas_nuevo = nuevos_datos[["State", "District ", "Crop", "Season"]]
X_numericas_nuevo = nuevos_datos[["Crop_Year", "Area "]]
X_categoricas_nuevo_procesada = encoder.transform(X_categoricas_nuevo)
X_nuevo_procesada = hstack([X_categoricas_nuevo_procesada, X_numericas_nuevo])

# Transformar a características polinomiales
X_nuevo_poly = poly.transform(X_nuevo_procesada)

# Realizar la predicción
prediccion = modelo.predict(X_nuevo_poly)

print("Predicción de Production:", abs(prediccion[0]))


Predicción de Production: 2449.9791536478733


In [ ]:
score = modelo.score(X_poly, y)
print(score)

0.9999995890925262
